#Assignment 2
#### Iheb Belgacem  2A CS Mai.2019
<br/>
Problem formulation : <br/>
We note N=size*size <br/>


* **Objective function** : 0  <br/>
* **Variables** : $$x_{i,j,k}=1~if~the~number~k~is~in~the~row~i~and~the~column~j~and~0~otherwise$$ 

* **Constraints**  
1. Each case in the Sudoku should have only one number : 

$$\sum_{k=1}^{N} x_{ijk} = 1~\forall~(i,j) \in \{1..N\}*\{1..N\}$$	
2. Each number is used once in each row <br/>
$$\sum_{i=1}^{N} x_{ijk} = 1~\forall~(j,k) \in \{1..N\}*\{1..N\}$$	
2. Each number is used once in each column <br/>
$$\sum_{j=1}^{N} x_{ijk} = 1~\forall~(i,k) \in \{1..N\}*\{1..N\}$$
4.Each number should be present once in every "group" :
$$\sum_{i=p}^{p+2}\sum_{j=q}^{q+2} x_{ijk} = 1~\forall~k \in \{1..N\}$$
$$ p~and~q~are~multiples~of~size,~strictly~smaller~than~N~$$

5. To express the numbers fixed from the beginning, we should introduce the following constraint :
$$x_{ijk} = 1$$
$$for~example~x_{115}=1 if~the~element~(1,1)~is~fixed~to~have~the~value~5 $$	




---

## The code :

In [0]:
import numpy as np
import cvxopt
import cvxopt.glpk
from cvxopt import matrix
import pandas as pd

In [0]:
size=3
n=size*size
#Initial Configuration of the sudoku table
Ai = np.array([
[0,8,0, 9,0,1, 0,5,0],
[0,0,2, 6,8,7, 3,0,0],
[0,0,3, 0,0,0, 6,0,0],
[3,9,0, 0,0,0, 0,6,5],
[6,0,0, 4,7,5, 0,0,3],
[5,7,0, 0,0,0, 0,8,4],
[0,0,9, 0,0,0, 8,0,0],
[0,0,5, 1,2,4, 9,0,0],
[0,4,0, 8,0,3, 0,2,0]])

In [140]:
ii=pd.DataFrame(Ai)
newcol=['|','|','|','|','|','|','|','|','|']
ii.insert(loc=3,column='O',value=newcol)
ii.insert(loc=7,column='o',value=newcol)
print ("intial configuration")
ii

intial configuration


,0,1,2,O,3,4,5,o,6,7,8
0,0,8,0,|,9,0,1,|,0,5,0
1,0,0,2,|,6,8,7,|,3,0,0
2,0,0,3,|,0,0,0,|,6,0,0
3,3,9,0,|,0,0,0,|,0,6,5
4,6,0,0,|,4,7,5,|,0,0,3
5,5,7,0,|,0,0,0,|,0,8,4
6,0,0,9,|,0,0,0,|,8,0,0
7,0,0,5,|,1,2,4,|,9,0,0
8,0,4,0,|,8,0,3,|,0,2,0


In [0]:
## Bijection from i,j,k to the index in the x-vector

def indice(i,j,k,size):
    n = size*size
    return(k+ j*n+ i*n*n)

In [0]:
## Take into account initial configuration
def initialconstraints(Ini,size):
  n=size*size
  nb=np.count_nonzero(Ini)
  A=np.zeros((nb,n*n*n))
  m=0
  for i in range(n):
    for j in range(n):
      if Ini[i,j]>0:
        A[m,indice(i,j,Ini[i,j]-1,size)]=1
        m=m+1
  return A

#line constraints
def lineconstraints(size):
  n=size*size
  m=0
  A=np.zeros((n*n,n*n*n))
  for j in range(n):
    for k in range(n):
      for i in range(n):
        A[m,indice(i,j,k,size)]=1
      m=m+1
  return A

#column constraints
def columnconstraints(size):
  n=size*size
  m=0
  A=np.zeros((n*n,n*n*n))
  for i in range(n):
    for k in range(n):
      for j in range(n):
        A[m,indice(i,j,k,size)]=1
      m=m+1
  return A
    

#column constraints
def columnconstraints(size):
  n=size*size
  m=0
  A=np.zeros((n*n,n*n*n))
  for i in range(n):
    for k in range(n):
      for j in range(n):
        A[m,indice(i,j,k,size)]=1
      m=m+1
  return A

#one number in every case
def kconstraint(size):
  n=size*size
  m=0
  A=np.zeros((n*n,n*n*n))
  for i in range(n):
    for j in range(n):
      for k in range(n):
        A[m,indice(i,j,k,size)]=1
      m=m+1
  return A    
    
  
#group constraints
def groupconstraints(size):
  m=0
  n=size*size
  A=np.zeros((n*n,n*n*n))
  for k in range(n):
    for p in range(size):
      for q in range(size):
        for ii in range (size):
          for jj in range(size):
            A[m,indice(size*p+ii,size*q+jj,k,size)]=1
        m=m+1
  return A

In [143]:
#Constraints Matrix
A=np.vstack((initialconstraints(Ai,size),lineconstraints(size),columnconstraints(size),groupconstraints(size),kconstraint(size)))
A.shape

(359, 729)

In [0]:
## solving
def solving(A):
  b=matrix(np.ones(A.shape[0])) 
  c=matrix(np.zeros(A.shape[1])) 
  G=matrix(np.zeros(A.shape))
  h=matrix(np.zeros(A.shape[0]))
  M=matrix(A)
  (status, solution) = cvxopt.glpk.ilp(c=c,G=G,h=h,A=M,b=b,B=set(range(A.shape[1])))
  return (status, solution)

In [0]:
(status,solution)=solving(A)

In [0]:
import pandas as pd 
def show(solution,size):
  n=size*size
  l=[]
  for ind in range(size*size):
    l.append(ind+1)
  df = pd.DataFrame(index=l,columns=l)
  for i in range(n):
    for j in range(n):
      for k in range(n):
        if solution[indice(i,j,k,size)]==1:
          df.loc[i+1,j+1]=k+1
  return df
df=show(solution,size)
newcol=['|','|','|','|','|','|','|','|','|']
df.insert(loc=3,column='O',value=newcol)
df.insert(loc=7,column='o',value=newcol)

In [147]:
print("Solution of the sudoku")
df

Solution of the sudoku


,1,2,3,O,4,5,6,o,7,8,9
1,7,8,6,|,9,3,1,|,4,5,2
2,4,5,2,|,6,8,7,|,3,1,9
3,9,1,3,|,5,4,2,|,6,7,8
4,3,9,4,|,2,1,8,|,7,6,5
5,6,2,8,|,4,7,5,|,1,9,3
6,5,7,1,|,3,6,9,|,2,8,4
7,2,3,9,|,7,5,6,|,8,4,1
8,8,6,5,|,1,2,4,|,9,3,7
9,1,4,7,|,8,9,3,|,5,2,6




---

### Let's find the result for the second table (with fewer intial elements)

In [0]:
Aih = np.array([
[7,0,0, 0,0,0, 4,0,0],
[0,2,0, 0,7,0, 0,8,0],
[0,0,3, 0,0,8, 0,0,9],
[0,0,0, 5,0,0, 3,0,0],
[0,6,0, 0,2,0, 0,9,0],
[0,0,1, 0,0,7, 0,0,6],
[0,0,0, 3,0,0, 9,0,0],
[0,3,0, 0,4,0, 0,6,0],
[0,0,9, 0,0,1, 0,0,5]])

In [149]:
iih=pd.DataFrame(Aih)
newcol=['|','|','|','|','|','|','|','|','|']
iih.insert(loc=3,column='O',value=newcol)
iih.insert(loc=7,column='o',value=newcol)
print ("intial configuration")
iih

intial configuration


,0,1,2,O,3,4,5,o,6,7,8
0,7,0,0,|,0,0,0,|,4,0,0
1,0,2,0,|,0,7,0,|,0,8,0
2,0,0,3,|,0,0,8,|,0,0,9
3,0,0,0,|,5,0,0,|,3,0,0
4,0,6,0,|,0,2,0,|,0,9,0
5,0,0,1,|,0,0,7,|,0,0,6
6,0,0,0,|,3,0,0,|,9,0,0
7,0,3,0,|,0,4,0,|,0,6,0
8,0,0,9,|,0,0,1,|,0,0,5


In [0]:
Ah=np.vstack((initialconstraints(Aih,size),lineconstraints(size),columnconstraints(size),groupconstraints(size),kconstraint(size)))
(statush,solutionh)=solving(Ah)
dfh=show(solutionh,size)
newcol=['|','|','|','|','|','|','|','|','|']
dfh.insert(loc=3,column='O',value=newcol)
dfh.insert(loc=7,column='o',value=newcol)

In [151]:
print("Solution of the harder sudoku")
dfh

Solution of the harder sudoku


,1,2,3,O,4,5,6,o,7,8,9
1,7,9,8,|,6,3,5,|,4,2,1
2,1,2,6,|,9,7,4,|,5,8,3
3,4,5,3,|,2,1,8,|,6,7,9
4,9,7,2,|,5,8,6,|,3,1,4
5,5,6,4,|,1,2,3,|,8,9,7
6,3,8,1,|,4,9,7,|,2,5,6
7,6,1,7,|,3,5,2,|,9,4,8
8,8,3,5,|,7,4,9,|,1,6,2
9,2,4,9,|,8,6,1,|,7,3,5




---

### Let's find the result for the special version (the computer scientist version)
*we have replaced the value 0 in the intial configuration given in the statement by G

In [0]:
size2=4
n2=size2*size2
#Initial Configuration of the sudoku table
Ais = np.array([
[8,15,0,12,0,0,0,0,0,10,0,0,0,0,0,6],
[0,0,0,10,0,0,0,15,0,0,0,11,7,4,13,0],
[11,0,4,0,0,0,13,6,0,7,0,0,16,0,5,0],
[1,0,0,0,0,0,0,16,3,0,9,2,0,0,0,0],
[0,0,0,0,0,1,15,13,0,3,16,0,0,14,7,4],
[0,1,0,6,0,0,0,12,0,11,0,0,10,0,3,0],
[0,12,0,13,0,0,6,3,0,5,0,0,9,2,0,0],
[9,0,3,4,14,0,2,0,0,0,7,13,0,0,0,0],
[0,0,0,0,5,7,0,0,0,8,0,12,3,16,0,10],
[0,0,14,2,0,0,4,0,7,1,0,0,15,0,6,0],
[0,5,0,3,0,0,8,0,9,0,0,0,14,0,12,0],
[7,16,6,0,0,12,9,0,13,14,3,0,0,0,0,0],
[0,0,0,0,13,14,0,4,16,0,0,0,0,0,0,2],
[0,7,0,8,0,0,12,0,4,2,0,0,0,11,0,5],
[0,2,9,14,11,0,0,0,5,0,0,0,4,0,0,0],
[6,0,0,0,0,0,7,0,0,0,0,0,1,0,8,3]
])

In [153]:
iis=pd.DataFrame(Ais)
newcol=['|','|','|','|','|','|','|','|','|','|','|','|','|','|','|','|']
iis.insert(loc=4,column='O',value=newcol)
iis.insert(loc=9,column='o',value=newcol)
iis.insert(loc=14,column='0',value=newcol)
print ("intial configuration")
iis.replace([10,11,12,13,14,15,16],['A','B','C','D','E','F','G'])

intial configuration


,0,1,2,3,O,4,5,6,7,o,8,9,10,11,0,12,13,14,15
0,8,F,0,C,|,0,0,0,0,|,0,A,0,0,|,0,0,0,6
1,0,0,0,A,|,0,0,0,F,|,0,0,0,B,|,7,4,D,0
2,B,0,4,0,|,0,0,D,6,|,0,7,0,0,|,G,0,5,0
3,1,0,0,0,|,0,0,0,G,|,3,0,9,2,|,0,0,0,0
4,0,0,0,0,|,0,1,F,D,|,0,3,G,0,|,0,E,7,4
5,0,1,0,6,|,0,0,0,C,|,0,B,0,0,|,A,0,3,0
6,0,C,0,D,|,0,0,6,3,|,0,5,0,0,|,9,2,0,0
7,9,0,3,4,|,E,0,2,0,|,0,0,7,D,|,0,0,0,0
8,0,0,0,0,|,5,7,0,0,|,0,8,0,C,|,3,G,0,A
9,0,0,E,2,|,0,0,4,0,|,7,1,0,0,|,F,0,6,0


In [158]:
As=np.vstack((initialconstraints(Ais,size2),lineconstraints(size2),columnconstraints(size2),groupconstraints(size2),kconstraint(size2)))
(statuss,solutions)=solving(As)
dfs=show(solutions,size2)
newcol=['|','|','|','|','|','|','|','|','|','|','|','|','|','|','|','|']
dfs.insert(loc=4,column='O',value=newcol)
dfs.insert(loc=9,column='o',value=newcol)
dfs.insert(loc=14,column='0',value=newcol)
print('the solution of 16*16 sudoku')
dfs.replace([10,11,12,13,14,15,16],['A','B','C','D','E','F','G'])


the solution of 16*16 sudoku


,1,2,3,4,O,5,6,7,8,o,9,10,11,12,0,13,14,15,16
1,8,F,G,C,|,1,B,5,7,|,E,A,D,4,|,2,3,9,6
2,3,6,2,A,|,8,9,E,F,|,1,G,5,B,|,7,4,D,C
3,B,E,4,9,|,2,3,D,6,|,F,7,C,8,|,G,A,5,1
4,1,D,5,7,|,C,4,A,G,|,3,6,9,2,|,B,8,F,E
5,2,B,8,5,|,9,1,F,D,|,A,3,G,6,|,C,E,7,4
6,E,1,F,6,|,7,8,G,C,|,2,B,4,9,|,A,5,3,D
7,G,C,7,D,|,4,A,6,3,|,8,5,1,E,|,9,2,B,F
8,9,A,3,4,|,E,5,2,B,|,C,F,7,D,|,8,6,1,G
9,4,9,D,1,|,5,7,B,E,|,6,8,F,C,|,3,G,2,A
10,C,8,E,2,|,3,G,4,A,|,7,1,B,5,|,F,D,6,9
